**IMPORTANT**

Change the Runtime version to 2025.07

Next to Conect or RAM & Disk, select more options and "Change runtime type"

# **A Notebook for Molecular Docking and Virtual Screening**

This notebook is based on Dr. Ashfaq Ahmad notebook for teaching purposes and it enables you to perform virtual screening for organic molecules on large virtual chemical libraries against the target protein of your choice using Autodock Vina 1.2.0 and PLIP 2.2.2.

If you are interested in learning this notebook in the form of training and learning such automation for other topics, please watch https://www.youtube.com/@Bioinformaticsinsights.

## List of the programs we will employ in this training

Firstly, we install all the necessary libraries and packages for virtual screening. The main s catg thates will be installed are:

+ AutoDock Vina (https://vina.scripps.edu/)
+ Biopython (https://biopython.org/)
+ Condacolab (https://github.com/conda-incubator/condacolab)
+ MGLtools (https://ccsb.scripps.edu/mgltools/)
+ OpenBabel (https://github.com/openbabel/openbabel)
+ PLIP (https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index)
+ Py3Dmol (https://pypict/py3Dmol).org/proje



In [ ]:
#@title **Installation of the required packages and dependencies**
#@markdown Thanks to **`mamba`**, the installation takes **less than 5 mins**. \
#@markdown It will **restart** the kernel (session).

%%capture
import contextlib
with open("/content/installation.log", "w") as i:
    with contextlib.redirect_stdout(i):

        !pip install numpy==1.26.4 # Added by JV for compatibility
        !pip install pandas==1.5.3 #
        !pip install py3Dmol==2.0.3
        !pip install pybel==0.15.5
        !pip install rdkit-pypi==2022.9.5

        !wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.0/vina_1.2.0_linux_x86_64 -O vina
        !chmod u+x vina

        !pip install condacolab
        import condacolab
        condacolab.install_mambaforge()

        !mamba install -c conda-forge -c bioconda mgltools=1.5.7 biopython=1.80 \
        openbabel=3.1.1 plip=2.2.2 zlib=1.2.13 xlsxwriter=3.0.3

        !rm -r /content/sample_data /content/condacolab_install.log

In [ ]:
#@title **Import Python modules**

import os
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')

import ast
import math
import plip
import timeit
import shutil
import py3Dmol
import contextlib
import xlsxwriter
import urllib.request

import numpy as np
import pandas as pd

from google.colab import drive, files
from tqdm.notebook import tqdm
from openbabel import pybel
from Bio.PDB import PDBIO, PDBParser
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, PandasTools
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex

print(f"+ Imported done")
print(f"+ Environment ready for molecular docking")

+ Imported done
+ Environment ready for molecular docking


In [ ]:
#@title **Import Google Drive**
#@markdown This allow data to be stored in Google Drive.

#with Hide():
drive.flush_and_unmount()
drive.mount("/content/drive", force_remount=True)

print(f"> Mounted at /content/drive")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
> Mounted at /content/drive


In [ ]:
#@title **Create folder**
#@markdown Enter a **< Job Name >** without space.\
#@markdown This creates a folder for protein, ligand, experimental, and docking.

Job_name = "4HEL" #@param {type:"string"}
assert not Job_name == "", "Do not leave this blank."
assert not any(c == "/" or c == "." for c in Job_name), "Disallowed characters."

DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, Job_name)
PRT_FLD = os.path.join(WRK_DIR, "PROTEIN")
LIG_FLD = os.path.join(WRK_DIR, "LIGAND")
EXP_FLD = os.path.join(WRK_DIR, "EXPERIMENTAL")
DCK_FLD = os.path.join(WRK_DIR, "DOCKING")
INT_FLD = os.path.join(WRK_DIR, "INTERACTION")

folders = [WRK_DIR, PRT_FLD, LIG_FLD, EXP_FLD, DCK_FLD, INT_FLD]

for f in folders:
    if os.path.exists(f):
        print(f"+ {os.path.basename(f)} folder already exists")
    if not os.path.exists(f):
        os.mkdir(f)
        print(f"+ {os.path.basename(f)} folder created")

+ 4HEL folder created
+ PROTEIN folder created
+ LIGAND folder created
+ EXPERIMENTAL folder created
+ DOCKING folder created
+ INTERACTION folder created


In [ ]:
#@title **Setting of utilities**
#@markdown This creates important variables and functions that will be utilized
#@markdown throughout the docking study.

%alias vina /content/vina

COLORS = ["red", "orange", "yellow", "lime", "green", "cyan", "teal", "blue",
          "violet", "purple", "pink", "gray", "brown", "white", "black"]

BOND_DICT = {"hydrophobic": ["0x59e382", "GREEN"],
             "hbond": ["0x59bee3", "LIGHT BLUE"],
             "waterbridge": ["0x4c4cff", "BLUE"],
             "saltbridge": ["0xefd033", "YELLOW"],
             "pistacking": ["0xb559e3", "PURPLE"],
             "pication": ["0xe359d8", "VIOLET"],
             "halogen": ["0x59bee3", "LIGHT BLUE"],
             "metal": ["0xe35959", "ORANGE"]}

# class Hide:
#     def __enter__(self):
#         self._original_stdout = sys.stdout
#         sys.stdout = open(os.devnull, "w")
#     def __exit__(self, exc_type, exc_val, exc_tb):
#         sys.stdout.close()
#         sys.stdout = self._original_stdout

class Hide:
    def __enter__(self):
        pass  # Do nothing when entering the 'with' block

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass  # Do nothing when exiting the 'with' block

# Biological assembly detector and extractor
def chain_profilier(inputFile):
    parser = PDBParser()
    structure = parser.get_structure("X", inputFile)
    chain_obj = [s for s in structure.get_chains()]
    chain_IDs = [c.get_id() for c in chain_obj]
    chain_len = len(chain_obj)
    return [chain_len, chain_IDs, chain_obj]

def seperate_chain(inputFile):
    io = PDBIO()
    name = os.path.basename(inputFile)[:-4]
    dir = os.path.dirname(inputFile)
    chain_info = chain_profilier(inputFile)
    chain_len, chain_IDs, chain_obj = chain_info[0], chain_info[1], chain_info[2]
    print(f"+ Chains detected: {chain_len} ({', '.join(chain_IDs)})")
    for ID,chain in zip(chain_IDs, chain_obj):
        chain_pdb = name + "_" + ID + ".pdb"
        chain_pdb_file = os.path.join(dir, chain_pdb)
        io.set_structure(chain)
        io.save(chain_pdb_file)
        print(f"+ {chain_pdb} ==> {os.path.basename(dir)} folder")
# Gridbox methods
def get_atom_coordinate(inputE):
    mol = Chem.MolFromPDBFile(inputE)
    molAtom = mol.GetAtoms()
    molConf = mol.GetConformer()
    xcoor = [molConf.GetAtomPosition(c).x for c in range(len(molAtom))]
    ycoor = [molConf.GetAtomPosition(c).y for c in range(len(molAtom))]
    zcoor = [molConf.GetAtomPosition(c).z for c in range(len(molAtom))]
    return [xcoor, ycoor, zcoor]

def labox(coorList, scale=2):
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    range = [[min(X), max(X)],
             [min(Y), max(Y)],
             [min(Z), max(Z)]]
    center = [np.round(np.mean(range[0]), 3),
              np.round(np.mean(range[1]), 3),
              np.round(np.mean(range[2]), 3)]
    size = [np.round(abs(range[0][0]-range[0][1])*scale, 3),
            np.round(abs(range[1][0]-range[1][1])*scale, 3),
            np.round(abs(range[2][0]-range[2][1])*scale, 3)]
    return [center, size]

def eboxsize(coorList, scale=1):
    gyBoxRatio = 0.23
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    center = [np.round(np.mean(X), 3),
              np.round(np.mean(Y), 3),
              np.round(np.mean(Z), 3),]
    dist = [(x-center[0])**2 +
            (y-center[1])**2 +
            (z-center[2])**2
            for x,y,z in zip(X,Y,Z)]
    size = [np.round((math.sqrt(np.sum(dist)/len(X))/gyBoxRatio)*scale, 3)]*3
    return [center, size]

def eboxsize_mod(coorList, scale=1):
    gyBoxRatio = 0.23
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    range = [[min(X), max(X)],
             [min(Y), max(Y)],
             [min(Z), max(Z)]]
    center = [np.round(np.mean(range[0]), 3),
              np.round(np.mean(range[1]), 3),
              np.round(np.mean(range[2]), 3)]
    dist = [(x-center[0])**2 +
            (y-center[1])**2 +
            (z-center[2])**2
            for x,y,z in zip(X,Y,Z)]
    size = [np.round((math.sqrt(np.sum(dist)/len(X))/gyBoxRatio)*scale, 3)]*3
    return [center, size]

def autodock_grid(coorList, scale=1):
    min_len = 22.5
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    range = [[min(X), max(X)],
             [min(Y), max(Y)],
             [min(Z), max(Z)]]
    center = [np.round(np.mean(range[0]), 3),
              np.round(np.mean(range[1]), 3),
              np.round(np.mean(range[2]), 3)]
    size = [min_len]*3
    return [center, size]

# Binding interactions profiler
def interaction_profiler(inputPL):
    protlig = PDBComplex()
    INTER = {}
    INTER_TYPE = []
    ID = os.path.basename(inputPL[:-7])
    CSV_Ifile = os.path.join(INT_FLD, f"{ID[:-2]}/{ID}_inter.csv")
    COL = ["RESNR", "RESTYPE", "RESCHAIN", "RESNR_LIG", "RESTYPE_LIG",
           "RESCHAIN_LIG", "DIST", "LIGCARBONIDX", "PROTCARBONIDX", "LIGCOO",
           "PROTCOO"]
    INTER_DF = pd.DataFrame(columns=COL)

    protlig.load_pdb(inputPL)
    LIG = protlig.ligands[0]
    protlig.characterize_complex(LIG)

    BD_keys = BOND_DICT.keys()
    BSI_key = f":".join([str(x) for x in LIG.members[0]])
    BSI_val = protlig.interaction_sets[BSI_key]
    BSR = BindingSiteReport(BSI_val)
    INTER[BSI_key] = {k: [getattr(BSR, f"{k}_features")] +
                      getattr(BSR, f"{k}_info")
                      for k in tuple(BD_keys)}
    for BD_key in list(BD_keys):
        DF = pd.DataFrame.from_records(
            INTER[BSI_key][BD_key][1:],
            columns=INTER[BSI_key][BD_key][0])
        if not DF.empty:
            INTER_TYPE.extend([BD_key.upper()]*len(DF))
            INTER_DF = pd.concat([INTER_DF, DF], ignore_index=True)
    INTER_DF = INTER_DF.assign(BOND=INTER_TYPE)
    INTER_DF.reset_index(drop=True, inplace=True)
    INTER_DF.to_csv(CSV_Ifile, index=False)

def view_interaction(inputCSV, result="summary"):
    DIST_CALC = []
    COLOR = []
    MIDCOO = []
    interaction = pd.read_csv(inputCSV, converters={
        "LIGCOO": lambda x: ast.literal_eval(str(x)),
        "PROTCOO": lambda x: ast.literal_eval(str(x)),
        "BOND": lambda x: x.lower()})
    for LC, PC, BT in zip(interaction["LIGCOO"], interaction["PROTCOO"],
                          interaction["BOND"]):
        COLOR.append(BOND_DICT[BT][1])
        p1 = np.array([LC[0], LC[1], LC[2]])
        p2 = np.array([PC[0], PC[1], PC[2]])
        squared_dist = np.sum((p1-p2)**2, axis=0)
        dist = np.round(np.sqrt(squared_dist) ,2)
        DIST_CALC.append(dist)
        mid_x = np.round((LC[0] + PC[0]) / 2, 2)
        mid_y = np.round((LC[1] + PC[1]) / 2, 2)
        mid_z = np.round((LC[2] + PC[2]) / 2, 2)
        p_mid = (mid_x, mid_y, mid_z)
        MIDCOO.append(p_mid)
    interaction["BOND"] = interaction["BOND"].str.upper()
    interaction["COLOR"] = COLOR
    interaction["MIDCOO"] = MIDCOO
    interaction["DIST_CALC"] = DIST_CALC
    interaction["RESNAME"] = interaction["RESTYPE"] + interaction["RESNR"].astype(str)

    summary = ["RESNR", "RESTYPE", "DIST_CALC", "BOND", "COLOR"]
    py3dmol = ["RESNR", "DIST_CALC", "LIGCOO", "PROTCOO", "MIDCOO", "BOND"]
    if result == "summary":
        df = interaction[summary]
    if result == "py3Dmol":
        df = interaction[py3dmol]
    return df

# Docking result clustering
def get_RMSD(ref, target):
    distances = []
    maxSubStr = rdFMCS.FindMCS([ref, target])
    a = ref.GetSubstructMatch(Chem.MolFromSmarts(maxSubStr.smartsString))
    b = target.GetSubstructMatch(Chem.MolFromSmarts(maxSubStr.smartsString))
    for atomA,atomB in list(zip(a, b)):
        pos_A = ref.GetConformer().GetAtomPosition(atomA)
        pos_B = target.GetConformer().GetAtomPosition(atomB)
        coord_A = np.array((pos_A.x, pos_A.y, pos_A.z))
        coord_B = np.array((pos_B.x, pos_B.y, pos_B.z))
        dist_numpy = np.linalg.norm(coord_A - coord_B)
        distances.append(dist_numpy)
    rmsd = np.round(math.sqrt(1/len(distances)*sum([i*i for i in distances])),3)
    return rmsd

def get_score(inputL, inputE, result="full"):
    ID = os.path.basename(inputL)[:-11]
    df = PandasTools.LoadSDF(inputL)
    df["ID"] = [ID + "_" + df.loc[i,"POSE"] for i in df.index]
    df["RMSD_EL"] = [get_RMSD(inputE, df.loc[i, "ROMol"]) for i in df.index]
    df["SCORE"] = pd.to_numeric(df["SCORE"])
    df["RMSD_EL"] = pd.to_numeric(df["RMSD_EL"])
    df["RMSD_LB_BP"] = pd.to_numeric(df["RMSD_LB_BP"])
    df["RMSD_UB_BP"] = pd.to_numeric(df["RMSD_UB_BP"])
    first = ["ID", "SCORE", "RMSD_EL"]
    full = ["ID", "POSE", "SCORE", "RMSD_EL", "RMSD_LB_BP", "RMSD_UB_BP"]
    if result == "first":
        return df[first].iloc[:1]
    if result == "full":
        return df[full]

# Py3Dmol model viewer
def VIEW_PROT(inputP, model="Cartoon", color="white", focusRes="",
              focusResColor="white", addStick=False, addLine=False,
              showChain=False, showResLabel=False, showVDW=False,
              outline=False):
    # Variables
    mview = py3Dmol.view(width=960, height=640)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein Model
    count = 0
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    mol = open(inputP, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {model: {cType: color}})

    # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showChain and chainLen > 1:
        for n,c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carbon"}})
            mview.addLabel(
                f"Chain {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": res}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": prot_model})
    mview.show()

def VIEW_ELIG(inputE, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(width=960, height=640)

    # Experimental ligand model
    count = 0
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    mol = open(inputE, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": elig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.zoomTo({"model": elig_model})
    mview.show()

def VIEW_LIG(inputL, showHs=False, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(width=960, height=640)

    # Ligand model
    count = 0
    lig_model = count
    print(f"+ Showing {os.path.basename(inputL)}")
    smi = open(inputL, "r").read()
    mol = Chem.MolFromSmiles(smi)
    #mol = Chem.AddHs(mol)
    mol = AllChem.AddHs(mol)
    AllChem.EmbedMolecule(mol)

    # Remove Hs
    if not showHs:
        mol = Chem.RemoveHs(mol)
    mblock = Chem.MolToMolBlock(mol)
    mview.addModel(mblock, "mol")
    mview.setStyle(
        {"model": lig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": lig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.show()

def VIEW_GRID(inputP, inputE, focusRes, center, size=[10, 10, 10]):
    # Variable
    mview = py3Dmol.view(width=960, height=640)

    # Grid box
    bxi, byi, bzi = center[0], center[1], center[2]
    bxf, byf, bzf = size[0], size[1], size[2]
    print(f"+ Center: X {center[0]}  Y {center[1]}  Z {center[2]}")
    print(f"+ Size: W {size[0]}  H {size[1]}  D {size[2]}")
    mview.addBox(
        {"center":{"x":bxi, "y":byi, "z":bzi},
        "dimensions": {"w":bxf, "h":byf, "d":bzf},
        "color": "skyBlue", "opacity": 0.7})

    # Protein model
    count = 0
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {"cartoon": {"color": "white"}})

    # Experimental ligand model
    count += 1
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    molB = open(inputE, "r").read()
    mview.addModel(molB, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "greenCarbon"}})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": "orangeCarbon", "radius": 0.15}})
        mview.addResLabels(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"alignment": "bottomLeft",
            "showBackground": False,
            "inFront": True,
            "fixed": False,
            "fontSize": 14,
            "fontColor": "0x000000",
            "screenOffset": {"x": 15, "y": 15}})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": elig_model})
    mview.show()

def VIEW_PILE(inputP, inputPP, inputL, inputE, inputCSV, model="Cartoon",
              color="white", focusRes="", focusResColor="white",
              showInter=False, showSubunit=False, showResLabel=False,
              showVDW=False, showPartProt=False, showExpLig=False,
              showLig=False, showAllPose=False, showBestPose=False,
              outline=False):
    # Variables
    mview = py3Dmol.view(width=960, height=640)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein model
    count = 0
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model":prot_model},
        {model: {cType: color}})

  # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showSubunit and chainLen > 1:
        for n, c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carboon"}})
            mview.addLabel(
                f"Subunit {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    res = focusRes.split(',')
    if focusRes != "":
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": res}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Interactions
    if showInter:
        interaction = view_interaction(inputCSV, result="py3Dmol")
        RESNR = interaction["RESNR"]
        DIST_CALC = interaction["DIST_CALC"]
        LIGCOO = interaction["LIGCOO"]
        PROTCOO = interaction["PROTCOO"]
        MIDCOO = interaction["MIDCOO"]
        BOND = interaction["BOND"]

        inter_res = list(set(RESNR) - set(res))
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": inter_res}]},
            {"stick": {"colorscheme": "whiteCarbon", "radius": 0.15}})
        mview.addResLabels(
            {"and": [{"model": prot_model}, {"resi": inter_res}]},
            {"alignment": "bottomLeft", "showBackground": False,
                "inFront": True, "fixed": False, "fontSize": 14,
                "fontColor": "0x000000", "screenOffset": {"x": 15, "y": 15}})

        for RN,DC,LC,PC,MC,BT in zip(RESNR, DIST_CALC, LIGCOO, PROTCOO, MIDCOO, BOND):
            BT = BT.lower()
            mview.addCylinder(
                {"start": {"x": LC[0], "y": LC[1], "z": LC[2]},
                    "end": {"x": PC[0], "y": PC[1], "z": PC[2]},
                    "radius": 0.05, "fromCap": 1, "toCap": 1,
                    "color": BOND_DICT[BT][0], "dashed": True})
            mview.addLabel(
                str(DC) + " Å",
                {"position": {"x": MC[0], "y": MC[1], "z": MC[2]},
                    "alignment": "bottomLeft", "inFront": False, "fixed": False,
                    "backgroundColor": BOND_DICT[BT][0], "fontSize": 10,
                    "screenOffset": {"x": 5, "y": 5}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Partner protein model
    if showPartProt and os.path.exists(inputPP):
        count += 1
        pprot_model = count
        print(f"+ Showing {os.path.basename(inputPP)} (yellow)")
        molB = open(inputPP, "r").read()
        mview.addModel(molB, "pdb")
        mview.setStyle(
            {"model": pprot_model},
            {"cartoon": {"color": "yellow"}})
        mview.addStyle(
            {"model": pprot_model},
            {"stick": {"colorscheme": "yellowCarbon"}})

    # Experimental ligand model
    if showExpLig:
        count += 1
        elig_model = count
        print(f"+ Showing {os.path.basename(inputE)} (gray)")
        molC = open(inputE, "r").read()
        mview.addModel(molC, "pdb")
        mview.setStyle(
            {"model": elig_model},
            {"stick": {"color": "gray"}})

    # Ligand model
    if showLig:
        count += 1
        lig_model = count
        print(f"+ Showing {os.path.basename(inputL)} (orange)")
        molD = open(inputL, "r").read()
        mview.addModel(molD, "pdb")
        mview.setStyle(
            {"model": lig_model},
            {"stick": {"colorscheme": "orangeCarbon"}})

    # Show poses of selected ligand
    if showAllPose:
        pose = sorted([ os.path.join(os.path.dirname(inputL), f) for f in os.listdir(os.path.dirname(inputL)) if f.endswith(".pdb") ])
        pose.remove(inputL[len(DCK_FLD)+1:]) if inputL[len(DCK_FLD)+1:] in pose else None
        for f in pose:
            count += 1
            pose_model = count
            fs = "".join(f)
            mol5 = open(os.path.join(DCK_FLD, fs), "r").read()
            mview.addModel(mol5, "pdb")
            mview.setStyle(
                {"model": pose_model},
                {"stick": {"color": "blue", "opacity": 0.5, "radius": 0.2}})

    # Show all best poses of docked ligands
    if showBestPose:
        pose = sorted([ f + "_1.pdb" for f in os.listdir(DCK_FLD) if os.path.isdir(os.path.join(DCK_FLD, f)) ])
        pose.remove(inputL[len(DCK_FLD)+1:]) if inputL[len(DCK_FLD)+1:] in pose else None
        for f in pose:
            count += 1
            pose_model = count
            fs = "".join(f)
            mol6 = open(os.path.join(DCK_FLD, f"{fs[:-6]}/{fs}"), "r").read()
            mview.addModel(mol6, "pdb")
            mview.setStyle(
                {"model": pose_model},
                {"stick": {"color": "green", "opacity": 0.5, "radius": 0.2}})

    if outline:
        mview.setViewStyle(
            {"style": "outline", "color": "black", "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.enableFog(True)
    mview.show()

# Preparing the Receptor molecules

Next, we download the protein crystal structure of interest in **`pdb`** file format from [Protein Data Bank (PDB)](https://www.rcsb.org/pdb) using an Accession ID.

In [ ]:
#@title **Upload protein PDB file**
#@markdown Click the **Choose Files** button below to upload your PDB file.

from google.colab import files
import shutil
import os

# Upload the PDB file
print("Please select your PDB file to upload:")
uploaded = files.upload()

# Get the uploaded filename
uploaded_filename = list(uploaded.keys())[0]

# Extract PDB_ID from the filename (assuming filename is like 5I3P.pdb or myprotein.pdb)
# Remove the .pdb extension to get the ID
PDB_ID = os.path.splitext(uploaded_filename)[0]

# Define the destination path
PDB_pdb_Pfile = os.path.join(PRT_FLD, uploaded_filename)

# Move the uploaded file to the protein folder
shutil.move(uploaded_filename, PDB_pdb_Pfile)

print(f"+ PDB uploaded: {uploaded_filename} ==> PROTEIN folder")
print(f"+ File location: {PDB_pdb_Pfile}")
print(f"+ PDB_ID set to: {PDB_ID}")

Please select your PDB file to upload:


Saving prot_lig_prod_nw.pdb to prot_lig_prod_nw.pdb
+ PDB uploaded: prot_lig_prod_nw.pdb ==> PROTEIN folder
+ File location: /content/4HEL/PROTEIN/prot_lig_prod_nw.pdb
+ PDB_ID set to: prot_lig_prod_nw


In [ ]:
PDB_ID

'prot_lig_prod_nw'

In [ ]:
# #@title **Extract protein**
# #@markdown This generate **clean** protein structure for molecular docking.\
# #@markdown The protein chains will be extracted into separate files.

# PDB_prot = PDB_ID + "_prot"
# PDB_prot_pdb = PDB_prot + ".pdb"
# PDB_prot_pdb_Pfile = os.path.join(PRT_FLD, PDB_prot_pdb)

# print(PDB_prot_pdb_Pfile)
# selected = ["ATOM"]
# with open(PDB_prot_pdb_Pfile, "w") as o, open(PDB_pdb_Pfile, "r") as i: # Use PDB_pdb_Pfile directly
#     for line in i:
#         row = line.split()
#         if any(a == row[0] for a in selected):
#             o.write(line)

# print(f"+ Protein extracted: {os.path.basename(PDB_pdb_Pfile)} --> {PDB_prot_pdb}") # Update print statement
# seperate_chain(PDB_prot_pdb_Pfile)



#@title **Extract protein**
#@markdown This generate **clean** protein structure for molecular docking.\
#@markdown The protein chains will be extracted into separate files.

PDB_prot = PDB_ID + "_prot"
PDB_prot_pdb = PDB_prot + ".pdb"
PDB_prot_pdb_Pfile = os.path.join(PRT_FLD, PDB_prot_pdb)

print(PDB_prot_pdb_Pfile)
selected = ["ATOM"]
with open(PDB_prot_pdb_Pfile, "w") as o, open(PDB_pdb_Pfile, "r") as i:
    for line in i:
        row = line.split()
        if row and any(a == row[0] for a in selected):  # Check if row is not empty
            o.write(line)

print(f"+ Protein extracted: {os.path.basename(PDB_pdb_Pfile)} --> {PDB_prot_pdb}")
seperate_chain(PDB_prot_pdb_Pfile)

/content/4HEL/PROTEIN/prot_lig_prod_nw_prot.pdb
+ Protein extracted: prot_lig_prod_nw.pdb --> prot_lig_prod_nw_prot.pdb
+ Chains detected: 1 (A)
+ prot_lig_prod_nw_prot_A.pdb ==> PROTEIN folder


In [ ]:
#@title **Display 3D protein** { run: "auto" }
#@markdown Enter the **< Protein >** to be viewed.
#View = "result_model_0_prot_A" #@param {type:"string"}
View = f"{PDB_prot}_A" #@param {type:"string"}
Model = "cartoon" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "" #@param {type:"string"}
Residues_colour = "orange" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_chains = True #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
pview_Pfile = os.path.join(PRT_FLD, f"{View}.pdb")

VIEW_PROT(inputP=pview_Pfile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showChain=Show_all_chains,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          outline=View_outline)

+ Showing prot_lig_prod_nw_prot_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Parameterise protein with Gasteiger charges**
#@markdown Enter the **< Protein >** to be parameterised.\
#@markdown This generate a **`protein.pdbqt`** file after:
#@markdown + Addition of Gasteiger Partial Charge
#@markdown + Addition of polar hydrogens
#@markdown + Removal of non-polar hydrogens

#Target_protein = "5I3P_prot_A" #@param {type:"string"}
Target_protein = f"{PDB_prot}_A" #@param {type:"string"}
PROT_pdb = Target_protein + ".pdb"
PROT_pdbqt = Target_protein + ".pdbqt"
PROT_pdb_Pfile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pdb_Dfile = os.path.join(DCK_FLD, PROT_pdb)
PROT_pdbqt_Dfile = os.path.join(DCK_FLD, PROT_pdbqt)

with Hide():
    #!prepare_receptor4.py -r $PROT_pdb_Pfile -o $PROT_pdbqt_Dfile -A hydrogens -U nphs_lps -v
    !python2 /usr/local/bin/prepare_receptor4.py -r $PROT_pdb_Pfile -o $PROT_pdbqt_Dfile -A hydrogens -U nphs_lps -v
    #!./vina --receptor $PROT_pdb_Pfile --prepare_receptor --out $PROT_pdbqt_Dfile
    shutil.copy(PROT_pdb_Pfile, PROT_pdb_Dfile)

print(f"+ Protein parameterised: {PROT_pdb} --> {PROT_pdbqt}")
print(f"+ {PROT_pdb} ==> DOCKING folder")
print(f"+ {PROT_pdbqt} ==> DOCKING folder")

set verbose to  True
read  /content/4HEL/PROTEIN/prot_lig_prod_nw_prot_A.pdb
setting up RPO with mode= automatic and outputfilename=  /content/4HEL/DOCKING/prot_lig_prod_nw_prot_A.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide
+ Protein parameterised: prot_lig_prod_nw_prot_A.pdb --> prot_lig_prod_nw_prot_A.pdbqt
+ prot_lig_prod_nw_prot_A.pdb ==> DOCKING folder
+ prot_lig_prod_nw_prot_A.pdbqt ==> DOCKING folder


# Extaction of the Experimental Ligand

We also will be extracting the experimental ligand for later comparison with the binding modes of docked liga We will redock it and will compare the binding poses, I call it a **Control Dock**.nds.

In [ ]:
print(PDB_pdb_Pfile)

/content/4HEL/PROTEIN/prot_lig_prod_nw.pdb


In [ ]:
#@title **Generate experimental ligand PDB file**
#@markdown Enter the **< Keyword >** assigned to query for the experimental
#@markdown ligand in the **`pdb`** fle.

Keyword = "UNK" #@param {type:"string"}
Elig_pdb = Keyword + ".pdb"
Elig_pdb_Efile = os.path.join(EXP_FLD, Elig_pdb)

selected = [Keyword]
with open(Elig_pdb_Efile, "w") as o, open(PDB_pdb_Pfile, "r") as i:
    for line in i:
        row = line.split()
        if any(a in row for a in selected):
            o.write(line)

print(f"+ Experimental ligand extracted: {PDB_ID}.pdb --> {Elig_pdb}") # Modified to use PDB_ID

+ Experimental ligand extracted: prot_lig_prod_nw.pdb --> UNK.pdb


In [ ]:
#@title **Extract experimental ligands**
#@markdown This generate **clean** experimental ligands for comparison.\
#@markdown The experimental ligands will be extracted into separate files.

seperate_chain(Elig_pdb_Efile)

+ Chains detected: 1 (A)
+ UNK_A.pdb ==> EXPERIMENTAL folder


In [ ]:
#@title **Display 3D ligand** {run: "auto"}
#@markdown Enter the **< Experimental Ligand >** to be viewed.

View = "UNK_A" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
eview_Efile = os.path.join(EXP_FLD, f"{View}.pdb")
#?
VIEW_ELIG(inputE=eview_Efile,
          showAtomLabel=Show_atom_labels,
          outline=View_outline)

+ Showing UNK_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Choose exprimental ligand**
#@markdown Enter the **< Target Experimental Ligand >** for later comparison.

Target_experimental_ligand = "UNK_A" #@param {type:"string"}
EXP_pdb = Target_experimental_ligand + ".pdb"
EXP_pdb_Efile = os.path.join(EXP_FLD, EXP_pdb)
EXP_pdb_Dfile = os.path.join(DCK_FLD, EXP_pdb)

with Hide():
    shutil.copy(EXP_pdb_Efile, EXP_pdb_Dfile)

print(f"+ {EXP_pdb} ==> DOCKING folder")

+ UNK_A.pdb ==> DOCKING folder


#  Ligand  Preparation for Docking

Now, we start to prepare ligands from a virtual chemical library for virtual scingreen The choice of library is yours, but majority of the people use compounds from **PubChem** and **Zinc**, but it is not compulsory. You can the use comps libraryound of your choing.

In [ ]:
# #@title **Provide chemical library**
# #@markdown Upload a **< Compound Library CSV >**  onto LIGAND folder.\
# #@markdown The file **should** contain the below informations:
# #@markdown + **`ID`** : The identity assigned on compound
# #@markdown + **`SMILES`** : The canonical / isometric SMILES notation

# import os
# from google.colab import files
# import shutil

# # Define Job_name variable (you should have this defined earlier in your notebook)
# # Job_name = "8P3D_VS_test_sep25"  # Example - this should be defined elsewhere

# # Create the LIGAND folder path
# LIG_FLD = os.path.join("/content", Job_name, "LIGAND")
# os.makedirs(LIG_FLD, exist_ok=True)

# print(f"📁 Upload your compound library CSV file")
# print(f"   Target folder: {LIG_FLD}")
# print("-" * 50)

# # Upload file
# try:
#     uploaded = files.upload()
# except Exception as e:
#     error_msg = f"File upload failed: {str(e)}"
#     print(f"❌ {error_msg}")
#     raise RuntimeError(error_msg)

# if uploaded:
#     # Get the uploaded filename (handles spaces, parentheses, etc.)
#     uploaded_filename = list(uploaded.keys())[0]
#     print(f"\n✅ Uploaded: {uploaded_filename}")

#     # Save the filename to the variable
#     Compound_library = uploaded_filename

#     # Define the full path where the file will be saved
#     CL_csv_Lfile = os.path.join(LIG_FLD, uploaded_filename)

#     # Write the uploaded content directly to the target location
#     with open(CL_csv_Lfile, 'wb') as f:
#         f.write(uploaded[uploaded_filename])
#     print(f"💾 Saved to: {CL_csv_Lfile}")

#     # Clean up: Remove the file from /content/ if it was created there
#     temp_uploaded_path = os.path.join("/content", uploaded_filename)
#     if os.path.exists(temp_uploaded_path) and os.path.abspath(temp_uploaded_path) != os.path.abspath(CL_csv_Lfile):
#         try:
#             os.remove(temp_uploaded_path)
#             print(f"🧹 Cleaned up temporary file from /content/")
#         except Exception as e:
#             print(f"⚠️  Could not remove temp file: {e}")

#     # Verify file exists in the correct location
#     if not os.path.exists(CL_csv_Lfile):
#         error_msg = f"Failed to save file to {CL_csv_Lfile}"
#         print(f"❌ {error_msg}")
#         raise FileNotFoundError(error_msg)

#     # Try to read the CSV with automatic delimiter detection
#     try:
#         # First, try to detect the delimiter
#         with open(CL_csv_Lfile, 'r') as f:
#             first_line = f.readline()
#             if ';' in first_line:
#                 delimiter = ';'
#                 print(f"🔍 Detected delimiter: semicolon (;)")
#             else:
#                 delimiter = ','
#                 print(f"🔍 Detected delimiter: comma (,)")

#         # Read the CSV with the detected delimiter
#         ligDF = pd.read_csv(CL_csv_Lfile, delimiter=delimiter, dtype=str)

#         # Check if required columns exist
#         required_cols = ["ID", "SMILES"]
#         existing_cols = ligDF.columns.tolist()

#         # Handle case-insensitive column matching
#         col_mapping = {}
#         for req_col in required_cols:
#             for exist_col in existing_cols:
#                 if req_col.lower() == exist_col.lower():
#                     col_mapping[exist_col] = req_col
#                     break

#         # Rename columns if needed
#         if col_mapping:
#             ligDF = ligDF.rename(columns=col_mapping)

#         # Select only ID and SMILES columns
#         if all(col in ligDF.columns for col in required_cols):
#             ligDF = ligDF[required_cols]
#             print(f"\n📊 Successfully loaded {len(ligDF)} compounds")
#             print(f"📋 Columns: {', '.join(ligDF.columns)}")
#             #print("\n" + "="*50)
#             #print("First 5 rows of the dataset:")
#             #print("="*50)
#             #display(ligDF.head())

#             # Show basic statistics
#             print("\n📈 Dataset Summary:")
#             print(f"   • Total compounds: {len(ligDF)}")
#             print(f"   • Unique IDs: {ligDF['ID'].nunique()}")
#             print(f"   • Non-null SMILES: {ligDF['SMILES'].notna().sum()}")

#             # Check for duplicates
#             duplicates = ligDF[ligDF.duplicated(subset=['ID'], keep=False)]
#             if not duplicates.empty:
#                 print(f"\n⚠️ Warning: Found {len(duplicates)} duplicate IDs")

#             # Ensure ligDF is properly formatted and accessible
#             print(f"\n✅ ligDF variable is ready with {len(ligDF)} compounds")

#         else:
#             missing_cols = [col for col in required_cols if col not in ligDF.columns]
#             error_msg = f"Missing required columns: {missing_cols}. Available columns: {ligDF.columns.tolist()}"
#             print(f"\n❌ Error: {error_msg}")
#             print("\n   Please ensure your CSV contains 'ID' and 'SMILES' columns")
#             raise ValueError(error_msg)

#     except Exception as e:
#         print(f"\n❌ Error reading CSV file: {str(e)}")
#         print("\n   Attempting alternative read methods...")

#         # Initialize ligDF as None
#         ligDF = None

#         # Try with different encodings and delimiters
#         for encoding in ['utf-8', 'latin-1', 'cp1252']:
#             for delim in [',', ';', '\t', '|']:
#                 try:
#                     temp_df = pd.read_csv(CL_csv_Lfile, delimiter=delim, encoding=encoding, dtype=str)

#                     # Check for required columns (case-insensitive)
#                     col_mapping = {}
#                     for req_col in ["ID", "SMILES"]:
#                         for exist_col in temp_df.columns:
#                             if req_col.lower() == exist_col.lower():
#                                 col_mapping[exist_col] = req_col
#                                 break

#                     if col_mapping:
#                         temp_df = temp_df.rename(columns=col_mapping)

#                     if 'ID' in temp_df.columns and 'SMILES' in temp_df.columns:
#                         ligDF = temp_df[["ID", "SMILES"]]
#                         print(f"   ✅ Successfully read with delimiter='{delim}' and encoding='{encoding}'")
#                         print(f"\n📊 Successfully loaded {len(ligDF)} compounds")
#                         display(ligDF.head())
#                         break
#                 except:
#                     continue
#             if ligDF is not None:
#                 break

#         if ligDF is None:
#             error_msg = f"Could not read the file '{uploaded_filename}' with any common delimiter/encoding combination. Please check your file format."
#             print(f"   ❌ {error_msg}")
#             raise ValueError(error_msg)
# else:
#     error_msg = "No file uploaded. Please upload a CSV file containing 'ID' and 'SMILES' columns."
#     print(f"❌ {error_msg}")
#     raise FileNotFoundError(error_msg)

# # Print the saved variable for verification
# if 'Compound_library' in locals() and Compound_library:
#     print(f"\n🏷️ Compound_library variable set to: '{Compound_library}'")
#     print(f"📂 Full path: {CL_csv_Lfile}")

# # Final check and display of ligDF
# print("\n" + "="*50)
# if 'ligDF' in locals() and ligDF is not None and not ligDF.empty:
#     print("✅ Final ligDF DataFrame:")
#     print("="*50)
#     display(ligDF)

#     # Output ligDF as the final cell output (same as original code)
#     ligDF
# else:
#     error_msg = "ligDF not properly loaded. Please check your CSV file format and ensure it contains 'ID' and 'SMILES' columns."
#     print(f"⚠️ {error_msg}")
#     raise RuntimeError(error_msg)

In [ ]:
#@title **Manually Create Compound Library**
#@markdown Enter your compounds below. Each line should have: ID,SMILES

# Default example data (you can edit this)
Compound_data = """
ZER,O=C1OC(CCCC(=O)CCCC=Cc2cc(O)cc(O)c12)C
"""
Compound_library_name = "ligand.csv" #@param {type:"string"}

import pandas as pd
import os

# Create the LIGAND folder path
LIG_FLD = os.path.join("/content", Job_name, "LIGAND")
os.makedirs(LIG_FLD, exist_ok=True)

print(f"📝 Creating compound library manually")
print(f"   Target folder: {LIG_FLD}")
print("-" * 50)

try:
    # Parse the input data
    lines = [line.strip() for line in Compound_data.strip().split('\n') if line.strip()]

    compounds = []
    for line in lines:
        parts = [p.strip() for p in line.split(',', 1)]  # Split only on first comma
        if len(parts) == 2:
            compounds.append({'ID': parts[0], 'SMILES': parts[1]})
        else:
            print(f"⚠️  Skipping invalid line: {line}")

    if not compounds:
        raise ValueError("No valid compounds found. Please check your input format.")

    # Create DataFrame
    ligDF = pd.DataFrame(compounds)

    # Save to CSV
    Compound_library = Compound_library_name
    CL_csv_Lfile = os.path.join(LIG_FLD, Compound_library)
    ligDF.to_csv(CL_csv_Lfile, index=False)

    print(f"\n✅ Created: {Compound_library}")
    print(f"💾 Saved to: {CL_csv_Lfile}")
    print(f"\n📊 Successfully created {len(ligDF)} compounds")
    print(f"📋 Columns: {', '.join(ligDF.columns)}")

    # Show basic statistics
    print("\n📈 Dataset Summary:")
    print(f"   • Total compounds: {len(ligDF)}")
    print(f"   • Unique IDs: {ligDF['ID'].nunique()}")
    print(f"   • Non-null SMILES: {ligDF['SMILES'].notna().sum()}")

    # Check for duplicates
    duplicates = ligDF[ligDF.duplicated(subset=['ID'], keep=False)]
    if not duplicates.empty:
        print(f"\n⚠️ Warning: Found {len(duplicates)} duplicate IDs")

    print(f"\n✅ ligDF variable is ready with {len(ligDF)} compounds")
    print(f"🏷️ Compound_library variable set to: '{Compound_library}'")
    print(f"📂 Full path: {CL_csv_Lfile}")

    # Display the dataframe
    print("\n" + "="*50)
    print("✅ Final ligDF DataFrame:")
    print("="*50)
    display(ligDF)

except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("\nPlease format your data as:")
    print("ID1,SMILES1")
    print("ID2,SMILES2")
    print("...")
    raise

#ligDF

📝 Creating compound library manually
   Target folder: /content/4HEL/LIGAND
--------------------------------------------------

✅ Created: ligand.csv
💾 Saved to: /content/4HEL/LIGAND/ligand.csv

📊 Successfully created 1 compounds
📋 Columns: ID, SMILES

📈 Dataset Summary:
   • Total compounds: 1
   • Unique IDs: 1
   • Non-null SMILES: 1

✅ ligDF variable is ready with 1 compounds
🏷️ Compound_library variable set to: 'ligand.csv'
📂 Full path: /content/4HEL/LIGAND/ligand.csv

✅ Final ligDF DataFrame:


,ID,SMILES
0,ZER,O=C1OC(CCCC(=O)CCCC=Cc2cc(O)cc(O)c12)C


In [ ]:
#@title **Generate SMILES files**
#@markdown This generate **`smi`** file for each ligands in the virtual library.

lig_IDs = tuple(ligDF["ID"])
lig_SMILESs = tuple(ligDF["SMILES"])

for N,ID in enumerate(lig_IDs):
    with open(os.path.join(LIG_FLD, f"{ID}.smi"), "w") as o:
        o.write(lig_SMILESs[N])

print(f"+ {N + 1} ligand.smi ==> LIGAND folder")

+ 1 ligand.smi ==> LIGAND folder


In [ ]:
#@title **Dispay 3D ligand** {run: "auto"}
#@markdown Enter the **< Ligand >** to be viewed.

View = "ZER" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
Show_hydrogens = True #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
lview_Lfile = os.path.join(LIG_FLD, f"{View}.smi")

VIEW_LIG(inputL=lview_Lfile,
         showHs=Show_hydrogens,
         showAtomLabel=Show_atom_labels,
         outline=View_outline)

+ Showing ZER.smi



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Robust Version: OpenBabel (3D) + prepare_ligand4.py (PDBQT)**

import os
import shutil
import subprocess
import time
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

# Force field selection for OpenBabel optimization
Select_force_field = "MMFF94" #@param ["MMFF94", "UFF", "Ghemical"]

# Use same DIR as original (for temporary files)
DIR = "/content/"

# Install OpenBabel if not present
if not shutil.which('obabel'):
    print("Installing OpenBabel...")
    os.system("apt-get update && apt-get install -y openbabel > /dev/null 2>&1")
    print("OpenBabel installed successfully!")

failed_ligands = []
successful_ligands = []

def generate_3d_mol2_openbabel(smiles, mol2_path, force_field="MMFF94", verbose=False):
    """
    Generate 3D MOL2 file from SMILES using OpenBabel.
    Returns True if successful, False otherwise.
    """
    temp_smi = f"{mol2_path}.tmp.smi"
    temp_mol2 = f"{mol2_path}.tmp.mol2"

    try:
        # Write SMILES to temp file
        with open(temp_smi, 'w') as f:
            f.write(smiles)

        # Method 1: Direct 3D generation with OpenBabel
        cmd = f"obabel -ismi {temp_smi} -omol2 -O {temp_mol2} --gen3D -h"
        if not verbose:
            cmd += " 2>/dev/null"

        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

        if result.returncode != 0 or not os.path.exists(temp_mol2):
            # Try alternative: generate SDF first, then convert
            temp_sdf = f"{mol2_path}.tmp.sdf"
            cmd2 = f"obabel -ismi {temp_smi} -osdf -O {temp_sdf} --gen3D -h"
            if not verbose:
                cmd2 += " 2>/dev/null"

            result = subprocess.run(cmd2, shell=True, capture_output=True, text=True)

            if result.returncode == 0 and os.path.exists(temp_sdf):
                # Convert SDF to MOL2
                cmd3 = f"obabel {temp_sdf} -O {temp_mol2}"
                if not verbose:
                    cmd3 += " 2>/dev/null"
                subprocess.run(cmd3, shell=True, capture_output=True, text=True)

                # Clean up SDF
                if os.path.exists(temp_sdf):
                    os.remove(temp_sdf)

        # Optimize with force field if MOL2 was created
        if os.path.exists(temp_mol2) and os.path.getsize(temp_mol2) > 0:
            # Optimize geometry
            cmd_opt = f"obabel {temp_mol2} -O {mol2_path} --ff {force_field} --minimize --steps 2500 --sd"
            if not verbose:
                cmd_opt += " 2>/dev/null"

            result = subprocess.run(cmd_opt, shell=True, capture_output=True, text=True)

            # If optimization fails, use unoptimized version
            if not os.path.exists(mol2_path) or os.path.getsize(mol2_path) == 0:
                shutil.copy(temp_mol2, mol2_path)

            # Clean up temp files
            for f in [temp_smi, temp_mol2]:
                if os.path.exists(f):
                    os.remove(f)

            return True

    except Exception as e:
        if verbose:
            print(f"OpenBabel error: {e}")

    # Clean up temp files on failure
    for f in [temp_smi, temp_mol2]:
        if os.path.exists(f):
            os.remove(f)

    return False

def validate_mol2(mol2_path):
    """
    Validate MOL2 file has proper structure and 3D coordinates.
    """
    if not os.path.exists(mol2_path):
        return False

    try:
        with open(mol2_path, 'r') as f:
            content = f.read()

        # Check for required MOL2 sections
        has_molecule = "@<TRIPOS>MOLECULE" in content
        has_atom = "@<TRIPOS>ATOM" in content
        has_bond = "@<TRIPOS>BOND" in content
        has_charges = "GASTEIGER" in content or "gasteiger" in content.lower()

        # Check if coordinates are 3D (Z values vary)
        import numpy as np
        z_coords = []
        in_atom_section = False

        for line in content.split('\n'):
            if "@<TRIPOS>ATOM" in line:
                in_atom_section = True
                continue
            elif "@<TRIPOS>" in line:
                in_atom_section = False
            elif in_atom_section and line.strip():
                parts = line.split()
                if len(parts) >= 6:
                    try:
                        z_coords.append(float(parts[4]))
                    except:
                        pass

        is_3d = len(z_coords) > 0 and np.std(z_coords) > 0.01

        return has_molecule and has_atom and has_bond and is_3d and has_charges

    except:
        return False

# Main processing loop
for N, ID in enumerate(tqdm(lig_IDs, desc="Processing ligands")):
    # Exact same paths as original code
    lig_Dfld = os.path.join(DCK_FLD, f"{ID}")
    os.makedirs(lig_Dfld, exist_ok=True)
    smi_file = os.path.join(LIG_FLD, f"{ID}.smi")
    mol2_file = os.path.join(DIR, f"{ID}.mol2")
    pdbqt_file = os.path.join(DCK_FLD, f"{ID}/{ID}.pdbqt")
    mol2_final = os.path.join(LIG_FLD, f"{ID}.mol2")

    # Remove existing files
    for f in [mol2_file, mol2_final, pdbqt_file]:
        if os.path.exists(f):
            os.remove(f)

    try:
        # Read SMILES
        with open(smi_file, 'r') as f:
            smiles = f.read().strip().split()[0]

        # Quick validation with RDKit (optional - just for checking)
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError(f"Invalid SMILES: {smiles}")

        # Generate 3D MOL2 with OpenBabel (robust method)
        success = generate_3d_mol2_openbabel(
            smiles=smiles,
            mol2_path=mol2_file,
            force_field=Select_force_field,
            verbose=False  # Set to True for debugging
        )

        if not success:
            raise ValueError("OpenBabel 3D generation failed")

        # Validate MOL2 file
        if not validate_mol2(mol2_file):
            raise ValueError("MOL2 validation failed - may be 2D or missing charges")

        # Use prepare_ligand4.py for PDBQT generation
        cmd_prepare = f"python2 /usr/local/bin/prepare_ligand4.py -l {mol2_file} -o {pdbqt_file} -U nphs_lps -q 2>/dev/null"
        result = subprocess.run(cmd_prepare, shell=True, capture_output=True, text=True)

        # Small delay to ensure file is written
        time.sleep(0.1)

        # Check if PDBQT exists (less strict checking)
        if not os.path.exists(pdbqt_file):
            # Try without -q flag (quiet mode) for better error messages
            cmd_prepare2 = f"python2 /usr/local/bin/prepare_ligand4.py -l {mol2_file} -o {pdbqt_file} -U nphs_lps"
            result2 = subprocess.run(cmd_prepare2, shell=True, capture_output=True, text=True)
            time.sleep(0.1)

            # If still fails, try with different parameters
            if not os.path.exists(pdbqt_file):
                # Try with all hydrogens and charges preserved
                cmd_prepare3 = f"python2 /usr/local/bin/prepare_ligand4.py -l {mol2_file} -o {pdbqt_file} -A hydrogens"
                subprocess.run(cmd_prepare3, shell=True, capture_output=True, text=True)
                time.sleep(0.1)

        # Final check
        if os.path.exists(pdbqt_file) and os.path.getsize(pdbqt_file) > 0:
            # Success! Move MOL2 to final location
            shutil.move(mol2_file, mol2_final)
            successful_ligands.append(ID)
        else:
            raise ValueError("PDBQT file is empty or not created")

    except Exception as e:
        # Try fallback method for difficult molecules
        try:
            print(f"\n⚠️ {ID}: Primary method failed ({str(e)[:50]}), trying fallback...")

            # Fallback: Use RDKit for simple molecules, OpenBabel for complex ones
            mol = Chem.MolFromSmiles(smiles)
            if mol and mol.GetRingInfo().NumRings() <= 2:
                # Simple molecule - try RDKit's 2D->3D
                from rdkit.Chem import AllChem, rdPartialCharges

                mol_h = Chem.AddHs(mol)
                AllChem.Compute2DCoords(mol_h)
                AllChem.EmbedMolecule(mol_h, randomSeed=42, useRandomCoords=False)

                # Quick optimization
                AllChem.UFFOptimizeMolecule(mol_h, maxIters=200)

                # Add charges
                rdPartialCharges.ComputeGasteigerCharges(mol_h)

                # Save as SDF and convert
                sdf_file = mol2_file.replace('.mol2', '.sdf')
                writer = Chem.SDWriter(sdf_file)
                writer.write(mol_h)
                writer.close()

                # Convert to MOL2
                cmd = f"obabel {sdf_file} -O {mol2_file} 2>/dev/null"
                os.system(cmd)

                # Clean up
                if os.path.exists(sdf_file):
                    os.remove(sdf_file)

                # Try PDBQT generation again
                if os.path.exists(mol2_file):
                    cmd_prepare = f"python2 /usr/local/bin/prepare_ligand4.py -l {mol2_file} -o {pdbqt_file} -U nphs_lps 2>/dev/null"
                    os.system(cmd_prepare)
                    time.sleep(0.1)

                    if os.path.exists(pdbqt_file) and os.path.getsize(pdbqt_file) > 0:
                        shutil.move(mol2_file, mol2_final)
                        successful_ligands.append(ID)
                        print(f"   ✓ {ID}: Fallback successful")
                        continue

        except:
            pass

        print(f"\n❌ Error with {ID}: {e}")
        failed_ligands.append(ID)
        continue

# Summary (matching original format)
print("\n" + "="*60)
print("PROCESSING SUMMARY")
print("="*60)
print(f"+ {len(successful_ligands)} ligand.mol2 ==> LIGAND folder")
print(f"+ {len(successful_ligands)} ligand.pdbqt ==> DOCKING folder")

if failed_ligands:
    print(f"\n⚠️ Failed molecules ({len(failed_ligands)}): {', '.join(failed_ligands)}")
    print("\nTroubleshooting for failed molecules:")
    print("1. Check if SMILES are valid")
    print("2. Try different force fields (UFF vs MMFF94)")
    print("3. Some molecules may need manual preparation in Avogadro/ChemDraw")
else:
    print("\n✅ All molecules processed successfully!")

# Optional: Show statistics for successful molecules
if successful_ligands:
    print("\n📊 Quick Statistics:")
    for i, ID in enumerate(successful_ligands[:3]):  # Show first 3
        mol2_path = os.path.join(LIG_FLD, f"{ID}.mol2")
        pdbqt_path = os.path.join(DCK_FLD, f"{ID}/{ID}.pdbqt")

        if os.path.exists(mol2_path) and os.path.exists(pdbqt_path):
            mol2_size = os.path.getsize(mol2_path)
            pdbqt_size = os.path.getsize(pdbqt_path)
            print(f"   {ID}: MOL2 ({mol2_size} bytes) → PDBQT ({pdbqt_size} bytes)")

    if len(successful_ligands) > 3:
        print(f"   ... and {len(successful_ligands) - 3} more")

Processing ligands: 100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


PROCESSING SUMMARY
+ 1 ligand.mol2 ==> LIGAND folder
+ 1 ligand.pdbqt ==> DOCKING folder

✅ All molecules processed successfully!

📊 Quick Statistics:
   ZER: MOL2 (4651 bytes) → PDBQT (2267 bytes)



# **Setting Up Molecular Docking**

are going to use Grid box and to hen define search space on the target pro box, whis ich often centered within the binding, active or allosteric site of target protein. The size should be sufficiently enough such that important binding residues are contained inside the box. Methods to define grid box include:

+ **`LaBOX`** = Utilize the min and max XYZ coordinates of (s).LaBOX)
+ **`eBoxSize`** = Utilize the radius of gyration of oxsize)
+ **`eBoxSize-modifed`** = Utilize `eBoxSize` but box center is calculated with `LABOGRID` method
+ **`Autodock-Grid`** = 22.5 × 22.5 × 22.5 Å
+ **`manual`** = Use the slider below to define grid box

In [ ]:
#@title **Place gridbox at binding site** { run: "auto" }
#@markdown Place the **< Gridbox >** at the center of binding site. \

Focus_residues = "" #@param {type:"string"}
Method = "LaBOX" #@param ["LaBOX", "eBoxSize", "eBoxSize-modified", "Autodock-Grid", "manual"]

#@markdown ---

if Method == "LaBOX":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = labox(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "eBoxSize":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = eboxsize(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "eBoxSize-modified":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = eboxsize_mod(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "Autodock-Grid":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = autodock_grid(atomCoord)
    Center, Size = grid[0], grid[1]

#@markdown **Manual Mode**
if Method == "manual":
    X = -2 #@param {type:"slider", min:-100, max:100, step:1}
    Y = -9 #@param {type:"slider", min:-100, max:100, step:1}
    Z = 9 #@param {type:"slider", min:-100, max:100, step:1}
    Width = 30 #@param {type:"slider", min:0, max:30, step:1}
    Height = 30 #@param {type:"slider", min:0, max:30, step:1}
    Depth = 30 #@param {type:"slider", min:0, max:30, step:1}
    Scale_factor = 1 #@param {type:"slider", min:0.1, max:5.0, step:0.1}
    Center = [X, Y, Z]
    Size = [Width*Scale_factor, Height*Scale_factor, Depth*Scale_factor]

VIEW_GRID(inputP=PROT_pdb_Dfile,
          inputE=EXP_pdb_Dfile,
          focusRes=Focus_residues,
          center=Center,
          size=Size)

+ Center: X -2.392  Y -4.386  Z -5.639
+ Size: W 9.054  H 16.674  D 15.608
+ Showing prot_lig_prod_nw_prot_A.pdb
+ Showing UNK_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Generate docking config file**
#@markdown This generate **Config File** for AutoDock Vina.

CONF = "config_file"
CONF_Dfile = os.path.join(DCK_FLD, CONF)

with open(CONF_Dfile, "w") as o:
    o.write(f"center_x = {Center[0]} \n")
    o.write(f"center_y = {Center[1]} \n")
    o.write(f"center_z = {Center[2]} \n")
    o.write(f"\n")
    o.write(f"size_x = {Size[0]} \n")
    o.write(f"size_y = {Size[1]} \n")
    o.write(f"size_z = {Size[2]} \n")

print(f"+ {CONF} ==> DOCKING folder" )

+ config_file ==> DOCKING folder


# **Perform Molecular Docking Job**

Finally, we are going to use Autodock Vina to perform molecular docking on each ligand contained in the chemical library. This operation mimics the virtual screening process. Each molecule may take longer than 2 minutes, depending on the number of rotatable bonds and level of exhaustiveness.

In [ ]:
#@title **Run AutoDock Vina**
#@markdown Select **< Exhaustiveness >** for molecular docking. (Default: 8)\
#@markdown Autodock Vina will performs molecular docking on targeted protein.\
#@markdown This generate **`ligand_output.pdbqt`**.

#@markdown **NOTE:** Compound with many rotatable bonds may required higher
#@markdown level of exhaustiveness to ensure accurate and consistent pose
#@markdown prediction.

Exhaustiveness = 24 #@param {type:"slider", min:1, max:128, step:1}
ex = Exhaustiveness
cpu_num = os.cpu_count()
time_elp = N = 0

for N,ID in enumerate(tqdm(lig_IDs)):
    pdbqt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}.pdbqt")
    log_txt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_log.txt")
    out_pdbqt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.pdbqt")

    # # -- Start docking --
    # start = timeit.default_timer()
    # with Hide():
    #     with open(log_txt_Dfile, "w") as i:
    #         with contextlib.redirect_stdout(i):
    #             %vina --receptor $PROT_pdbqt_Dfile --ligand $pdbqt_Dfile \
    #             --out $out_pdbqt_Dfile --config $CONF_Dfile \
    #             --exhaustiveness $ex --cpu $cpu_num --verbosity 2
    #     with open(log_txt_Dfile, "r") as r:
    #         data = r.read().splitlines(True)
    #     with open(log_txt_Dfile, "w") as o:
    #         o.writelines(data[1:])
    # stop = timeit.default_timer()
    # # -- End docking --

    # -- Start docking --
    start = timeit.default_timer()
    with open(log_txt_Dfile, "w") as i:
      with contextlib.redirect_stdout(i):
        %vina --receptor $PROT_pdbqt_Dfile --ligand $pdbqt_Dfile \
        --out $out_pdbqt_Dfile --config $CONF_Dfile \
        --exhaustiveness $ex --cpu $cpu_num --verbosity 2
    with open(log_txt_Dfile, "r") as r:
      data = r.read().splitlines(True)
    with open(log_txt_Dfile, "w") as o:
      o.writelines(data[1:])
    stop = timeit.default_timer()
    # -- End docking --

    runtime = stop - start
    time_elp += runtime
    avg = int(time_elp/(N+1))
    remain = int(avg*(len(lig_IDs)-N)-avg)

    RTO = f"{np.round(runtime, 2)}s"
    RO = f"{remain//3600:02}:{remain%3600//60:02}:{remain%60:02}"
    print(f"{ID:^10} {f'{RTO} taken':^20}  {f'{RO} remaining':^24}")

print(f"")
print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")

100%|██████████| 1/1 [00:38<00:00, 38.61s/it]

   ZER         38.61s taken         00:00:00 remaining   

+ Operation completed
+ Time elapsed: 00m 38s


In [ ]:
#@title **Process output file**
#@markdown This generate **`ligand_output.sdf`** for comparison with experimental ligand.\
#@markdown This also generate **`ligand.pdb`** for each binding modes.

pose_count = 0
for N,ID in enumerate(tqdm(lig_IDs)):
    out_pdbqt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.pdbqt")
    out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")
    out_d_pdb_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_.pdb")
    input = pybel.readfile("pdbqt", out_pdbqt_Dfile)
    output = pybel.Outputfile("sdf", out_sdf_Dfile, True)

    mols = [m for m in input]
    for mol in mols:
        mol.data.update({"POSE": mol.data["MODEL"]})
        mol.data.update({"SCORE": mol.data["REMARK"].split()[2]})
        mol.data.update({"RMSD_LB_BP": mol.data["REMARK"].split()[3]})
        mol.data.update({"RMSD_UB_BP": mol.data["REMARK"].split()[4]})
        del mol.data["MODEL"], mol.data["REMARK"], mol.data["TORSDO"]
        output.write(mol)
    output.close()

    with Hide():
        !obabel $out_pdbqt_Dfile -O $out_d_pdb_Dfile -m

    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    pose_count += len(lig_files)-4

print(f"+ {N+1} ligand_output.sdf ==> DOCKING folder")
print(f"+ {pose_count} ligand_n.pdb ==> DOCKING folder")

  0%|          | 0/1 [00:00<?, ?it/s]

9 molecules converted
9 files output. The first is /content/4HEL/DOCKING/ZER/ZER_1.pdb


100%|██████████| 1/1 [00:00<00:00,  7.87it/s]

+ 1 ligand_output.sdf ==> DOCKING folder
+ 9 ligand_n.pdb ==> DOCKING folder



# Calculate binding interactions throghgPLIP*

We will use PLIP to determine the binding interaction between the docked ligands and target proteins.

In [ ]:
#@title **Generate protein-ligand complex file**
#@markdown This merge ligand pose and protein into a **`PL.pdb`** file. \
#@markdown Required by **PLIP** to profile binding interactions.

# N = 0
# for ID in tqdm(lig_IDs):
#     lig_files = os.listdir(os.path.join(DCK_FLD, ID))
#     lig_Ifld = os.path.join(INT_FLD, f"{ID}")
#     os.makedirs(lig_Ifld, exist_ok=True)

#     for n in range(len(lig_files)-4):
#         pose_pdb_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_{n+1}.pdb")
#         complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")
#         prot_data = open(PROT_pdb_Dfile, "r").readlines()
#         pose_data = open(pose_pdb_Dfile, "r").readlines()

#         with open(complex_pdb_Ifile, "w") as o:
#             for line in prot_data[:-1]:
#                 o.write(line)
#             for line in pose_data:
#                 word = line.split()
#                 if word[0] == "ATOM" or word[0] == "CONECT":
#                     o.write(line)
#             o.write("END")
#         N += 1

# print(f"+ {N} PL.pdb ==> INTERACTION folder")



N = 0
for ID in tqdm(lig_IDs):
    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    lig_Ifld = os.path.join(INT_FLD, f"{ID}")
    os.makedirs(lig_Ifld, exist_ok=True)

    for n in range(len(lig_files) - 4):
        pose_pdb_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_{n+1}.pdb")
        complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")
        prot_data = open(PROT_pdb_Dfile, "r").readlines()

        # Check if the pose file exists before processing
        if not os.path.exists(pose_pdb_Dfile):
            print(f"Warning: {pose_pdb_Dfile} does not exist. Skipping.")
            continue

        pose_data = open(pose_pdb_Dfile, "r").readlines()

        with open(complex_pdb_Ifile, "w") as o:
            for line in prot_data[:-1]:
                o.write(line)
            for line in pose_data:
                word = line.split()
                if word[0] == "ATOM" or word[0] == "CONECT":
                    o.write(line)
            o.write("END")
        N += 1

print(f"+ {N} PL.pdb ==> INTERACTION folder")

100%|██████████| 1/1 [00:00<00:00, 93.07it/s]

+ 9 PL.pdb ==> INTERACTION folder


In [ ]:
#@title **Profile protein-ligand interactions**
#@markdown This identify **binding interactions** between docked ligand and target protein. \
#@markdown All interactions data is exported as **`inter.csv`** file.

# N = 0
# for ID in tqdm(lig_IDs):
#     lig_files = os.listdir(os.path.join(DCK_FLD, ID))
#     for n in range(len(lig_files)-4):
#         complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")
#         interaction_profiler(complex_pdb_Ifile)
#         N += 1

# print(f"+ {N} inter.CSV ==> INTERACTION folder")



N = 0
for ID in tqdm(lig_IDs):
    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    for n in range(len(lig_files) - 4):
        complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")

        # Ensure the directory for the complex file exists
        output_dir = os.path.dirname(complex_pdb_Ifile)
        os.makedirs(output_dir, exist_ok=True)

        # Check if the input file exists
        if not os.path.exists(complex_pdb_Ifile):
            print(f"Warning: {complex_pdb_Ifile} does not exist. Skipping.")
            continue

        # Check if the interaction_profiler's output directory exists
        profiler_output_dir = os.path.join(INT_FLD, ID)
        os.makedirs(profiler_output_dir, exist_ok=True)

        # Ensure the profiler works with correct paths
        try:
            interaction_profiler(complex_pdb_Ifile)
        except OSError as e:
            print(f"Error processing {complex_pdb_Ifile}: {e}")
            continue

        N += 1

print(f"+ {N} inter.CSV ==> INTERACTION folder")

100%|██████████| 1/1 [00:02<00:00,  2.83s/it]

+ 9 inter.CSV ==> INTERACTION folder


# Visualization of Docking Results**

The final step is to evaluate the docking score and visualize the 3D binding mode. Note thathe t docking score is essentially predicted binding affinity.

In [ ]:
#@title **Generate docking results**
#@markdown This generate **scoring result** of docked ligands in **`csv`** format.

EXP_pose = Chem.MolFromPDBFile(EXP_pdb_Dfile)
EXP_pose.SetProp("_Name", os.path.basename(EXP_pdb_Dfile)[:-4])

for N,ID in enumerate(tqdm(lig_IDs)):
    out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")
    rank_csv_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_result.csv")
    data = pd.DataFrame(get_score(out_sdf_Dfile, EXP_pose, result="full"))
    data.to_csv(rank_csv_Dfile, index=False)

print(f"+ {N+1} result.csv ==> DOCKING folder")

100%|██████████| 1/1 [00:00<00:00,  4.46it/s]

+ 1 result.csv ==> DOCKING folder


In [ ]:
#@title **Rank docking scores**
#@markdown This cluster and rank **scoring results** in ascending order. \
#@markdown The overall ranking is exported as **`ranked_result.csv`**. \
#@markdown **Top 10** results are shown below. \

#@markdown Noted that:
#@markdown + **`RMSD_EL`** = RMSD vs Exp. Lig.
#@markdown + **`RMSD_LB_BP`** = RMSD Lower Bound vs Best Pose
#@markdown + **`RMSD_LB_UP`** = RMSD Upper Bound vs Best Pose

# vina_rank = pd.DataFrame()
# for N,ID in enumerate(lig_IDs):
#     out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")
#     vina_rank = pd.concat([vina_rank, get_score(out_sdf_Dfile, EXP_pose, result="first")])

# vina_rank = vina_rank.sort_values("SCORE", ascending=True, ignore_index=True)
# rank_csv_Dfile = os.path.join(DCK_FLD, "ranked_result.csv")
# vina_rank.to_csv(rank_csv_Dfile)

# print(f"+ 1 ranked_result.xlsx ==> DOCKING folder")
# vina_rank

vina_rank = pd.DataFrame()
for N, ID in enumerate(lig_IDs):
    out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")

    # Check if the file exists
    if not os.path.exists(out_sdf_Dfile):
        print(f"Warning: {out_sdf_Dfile} does not exist. Skipping.")
        continue

    try:
        # Check the output of get_score
        score_data = get_score(out_sdf_Dfile, EXP_pose, result="first")
        if "SCORE" not in score_data.columns:
            print(f"Error: 'SCORE' column missing in output for {out_sdf_Dfile}. Skipping.")
            continue
        vina_rank = pd.concat([vina_rank, score_data])
    except Exception as e:
        print(f"Error processing {out_sdf_Dfile}: {e}")
        continue

# Ensure sorting works only if vina_rank is not empty
if not vina_rank.empty:
    vina_rank = vina_rank.sort_values("SCORE", ascending=True, ignore_index=True)
    rank_csv_Dfile = os.path.join(DCK_FLD, "ranked_result.csv")
    vina_rank.to_csv(rank_csv_Dfile)
    print(f"+ 1 ranked_result.csv ==> DOCKING folder")
else:
    print("No valid scores were found to rank.")

vina_rank


+ 1 ranked_result.csv ==> DOCKING folder


,ID,SCORE,RMSD_EL
0,ZER_1,-9.327,6.926


In [ ]:
#@title **Show docking result of ligand** {run : "auto"}

Select_ligand = "ZER" #@param {type:"string"}
LIG_out_csv_Dfile = os.path.join(DCK_FLD, f"{Select_ligand}/{Select_ligand}_result.csv")
ligResultDF = pd.read_csv(LIG_out_csv_Dfile)
ligResultDF

,ID,POSE,SCORE,RMSD_EL,RMSD_LB_BP,RMSD_UB_BP
0,ZER_1,1,-9.327,6.926,0.000,0.000
1,ZER_2,2,-8.465,5.426,2.096,4.438
2,ZER_3,3,-8.402,2.393,2.711,7.820
3,ZER_4,4,-7.884,3.081,1.895,6.811
4,ZER_5,5,-7.716,4.187,1.544,5.964
5,ZER_6,6,-7.533,4.784,1.993,5.452
6,ZER_7,7,-7.259,4.322,1.550,5.731
7,ZER_8,8,-7.231,5.485,2.359,5.193
8,ZER_9,9,-7.207,1.568,1.783,7.132


In [ ]:
#@title **Show interaction profile of ligand** {run: "auto"}

Select_ligand = "ZER_1" #@param {type : "string"}
LIG_inter_CSV_Ifile = os.path.join(INT_FLD, f"{Select_ligand[:-2]}/{Select_ligand}_inter.csv")
view_interaction(LIG_inter_CSV_Ifile, result="summary")

,RESNR,RESTYPE,DIST_CALC,BOND,COLOR
0,14,LEU,3.60,HYDROPHOBIC,GREEN
1,55,LEU,3.68,HYDROPHOBIC,GREEN
2,65,ILE,3.80,HYDROPHOBIC,GREEN
3,87,LEU,3.16,HYDROPHOBIC,GREEN
4,91,ARG,3.78,HYDROPHOBIC,GREEN
5,145,LEU,3.44,HYDROPHOBIC,GREEN
6,138,LEU,3.46,HYDROPHOBIC,GREEN
7,138,LEU,3.49,HYDROPHOBIC,GREEN
8,59,ARG,3.43,HBOND,LIGHT BLUE


In [ ]:
#@title **Display 3D binding mode** { run: "auto" }

#@markdown **PROTEIN MODEL**
#View_protein = "5I3P_prot_A" #@param {type:"string"}
View_protein = f"{PDB_prot}_A" #@param {type:"string"}
Model = "cartoon" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "" #@param {type:"string"}
Residues_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_subunits = False #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_interaction = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}

#@markdown ---

#@markdown **LIGAND MODEL** \
#@markdown Enter the **< Ligand >** to be viewed.
View_ligand = "ZER_1" #@param {type:"string"}
View_experimental_ligand = "UNK_A" #@param {type:"string"}
Show_ligand = True #@param {type:"boolean"}
Show_experimental_ligand = True #@param {type:"boolean"}
Show_all_pose = False #@param {type:"boolean"}
Show_best_pose = False #@param {type:"boolean"}

#@markdown ---

#@markdown **PARTNER PROTEIN MODEL** \
#@markdown (OPTIONAL, upload to DOCKING FOLDER) \
#@markdown Enter the **< Partner Protein>** to be view.

View_partner_protein = "" #@param {type:"string"}
Show_partner_protein = False #@param {type:"boolean"}

pview_Dfile = os.path.join(DCK_FLD, f"{View_protein}.pdb")
ppview_Dfile = os.path.join(DCK_FLD, f"{View_partner_protein}.pdb")
lview_Dfile = os.path.join(DCK_FLD, f"{View_ligand[:-2]}/{View_ligand}.pdb")
eview_Dfile = os.path.join(DCK_FLD, f"{View_experimental_ligand}.pdb")
iview_Ifile = os.path.join(INT_FLD, f"{View_ligand[:-2]}/{View_ligand}_inter.csv")

VIEW_PILE(inputP=pview_Dfile,
          inputPP=ppview_Dfile,
          inputL=lview_Dfile,
          inputE=eview_Dfile,
          inputCSV=iview_Ifile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showInter=Show_interaction,
          showSubunit=Show_all_subunits,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          showPartProt=Show_partner_protein,
          showExpLig=Show_experimental_ligand,
          showLig=Show_ligand,
          showAllPose=Show_all_pose,
          showBestPose=Show_best_pose,
          outline=View_outline)

+ Showing prot_lig_prod_nw_prot_A.pdb
+ Showing UNK_A.pdb (gray)
+ Showing ZER_1.pdb (orange)



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Save Resulte to Google drive OR Download manually
Once you download your results, you can prepare figures of your choice using Pymol.
+ In case Pymol is not installed on your system, you can find the whole installation tutorial here. **https://youtu.be/2pnwMS5JKis?si=a35chh59jOEv36TD**.
+  If you have Pymol, but don't know how to use it for figure generation, please follow this tutorial. **https://youtu.be/frnfgRZzcKs?si=tDFmrVrCIdF0KCQC**.ive.

In [ ]:
# #@title **Import Google Drive**
# #@markdown This allow data to be stored in Google Drive.

# with Hide():
#   drive.flush_and_unmount()
#   drive.mount("/content/drive", force_remount=True)

# print(f"> Mounted at /content/drive")

In [ ]:
#@title **Store result in Google Drive**
#@markdown Enter the **< Destination Path >** for saving. \
#@markdown This save all docking data into Google Drive.

Destination = "/content/drive/MyDrive/Biosensores/Manuscript/Nat_prots/Docking_Vina" # @param {type:"string"}
DST_FLD = os.path.join(Destination, Job_name)
shutil.copytree(WRK_DIR, DST_FLD)

print(f"> Data saved at {DST_FLD}")

> Data saved at /content/drive/MyDrive/Biosensores/Manuscript/Nat_prots/Docking_Vina/21_4


## **Congratulations**
You have completed this training. Now you are able to perform docking and virtual screening.